### Imports

In [1]:
from breeze_connect import BreezeConnect
from jproperties import Properties
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from pandas_market_calendars import get_calendar
import opstrat as op
import math

### Utility Methods

In [2]:
def get_number_of_trading_sessions(start_date, end_date):
    # Get the NSE calendar
    nse = get_calendar('NSE')

    # Get the trading sessions between the start and end dates
    trading_sessions = nse.schedule(start_date=start_date, end_date=end_date)

    # Return the number of trading sessions
    return len(trading_sessions)

assert(get_number_of_trading_sessions('2023-06-26', '2023-07-02') == 4)



In [3]:
getDF = lambda data : pd.DataFrame(data["Success"])

### Login to Breeze API

In [4]:
configs = Properties()
with open('app.properties', 'rb') as read_prop:
    configs.load(read_prop,"utf-8")
if len(configs) == 0:
    print("Error while reading properties files")

api_key = configs.get("APP_KEY").data if configs.get("APP_KEY") else None
api_secret = configs.get("SECRET_KEY").data if configs.get("SECRET_KEY") else None
session_token =configs.get("SESSION_TOKEN").data if configs.get("SESSION_TOKEN") else None
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret,session_token=session_token)

### Some constants

In [5]:
exchange_code = "NSE"

### get range of nifty based on trade

In [6]:
# get trading sessions
start_date = '2024-03-08'
end_date = '2024-03-31'
num_trading_sessions = get_number_of_trading_sessions(start_date, end_date)
print("Number of trading sessions is ", num_trading_sessions)

Number of trading sessions is  16


### get nifty value

In [7]:
df = getDF( breeze.get_quotes("NIFTY","NSE"))
df=df[df['exchange_code'] == exchange_code]
nifty_value=df['ltp'][0]
print("Nifty ",nifty_value)

Nifty  22493.55


In [16]:
breeze.get_names("NSE","ACC")

{'exchange_code': 'NSE',
 'exchange_stock_code': 'ACC',
 'isec_stock_code': 'ACC',
 'isec_token': '22',
 'company name': 'ACC LIMITED',
 'isec_token_level1': '4.1!22',
 'isec_token_level2': '4.2!22'}

In [14]:
breeze.get_customer_details(session_token)

{'Success': {'exg_trade_date': {'NSE': '11-Mar-2024',
   'BSE': '11-Mar-2024',
   'FNO': '11-Mar-2024',
   'NDX': '11-Mar-2024'},
  'exg_status': {'NSE': 'C', 'BSE': 'C', 'FNO': 'Y', 'NDX': 'C'},
  'segments_allowed': {'Trading': 'Y',
   'Equity': 'Y',
   'Derivatives': 'Y',
   'Currency': 'D'},
  'idirect_userid': 'AG147240',
  'idirect_user_name': 'BHUSHAN SHAM DHUMAL',
  'idirect_ORD_TYP': '',
  'idirect_lastlogin_time': '09-Mar-2024 20:25:47',
  'mf_holding_mode_popup_flg': 'N',
  'commodity_exchange_status': 'Y',
  'commodity_trade_date': '11-Mar-2024',
  'commodity_allowed': 'C'},
 'Status': 200,
 'Error': None}

### get india vix

In [40]:
stock_code = "INDVIX"
t = breeze.get_quotes(stock_code=stock_code,
                    exchange_code="NSE")
df = getDF(t)
df=df[df['exchange_code'] == exchange_code]
india_vix=df['ltp'][0]
# df=df[df['exchange_code'] == exchange_code]
india_vix

13.6125

In [43]:
print("VIX ",india_vix)

# calculation
daily_vix = india_vix / math.sqrt(252)
period_vix = daily_vix*num_trading_sessions
print("period_vix ",period_vix)
low = round(nifty_value*(100-period_vix)/100,2)
high = round(nifty_value*(100+period_vix)/100,2)
nifty_range = round(high - low,2)
print("Low ",low)
print("High ",high)
print("Premium ",nifty_range)

VIX  13.6125
period_vix  13.720110370234949
Low  19407.41
High  25579.69
Premium  6172.28


### get_portfolio_holdings

In [13]:
getDF(breeze.get_portfolio_holdings(exchange_code="NSE",
                                from_date="2022-08-01T06:00:00.000Z",
                                to_date="2022-08-19T06:00:00.000Z",
                                stock_code="",
                                portfolio_type=""))

,stock_code,exchange_code,quantity,average_price,booked_profit_loss,current_market_price,change_percentage,answer_flag,product_type,expiry_date,strike_price,right,category_index_per_stock,action,realized_profit,unrealized_profit,open_position_value,portfolio_charges
0,GOLDEX,NSE,59,53,0,55.44,1.31578947368421,N,None,None,None,None,None,None,None,None,None,None
1,NIFBEE,NSE,7,203.49,0,248.67,0.0201110127906041,N,None,None,None,None,None,None,None,None,None,None
2,INDPAI,NSE,5,1525.89,0,1363.55,-0.774996361519429,N,None,None,None,None,None,None,None,None,None,None
3,ITC,NSE,30,446.4,0,413.55,1.3975726369989,N,None,None,None,None,None,None,None,None,None,None


### Historic Data

In [17]:
# For equity
interval = "1day" # "1minute", "5minute", "30minute" or as "1day"
from_date = "17-Jun-2013"
to_date = "06-Jul-2023"
stock_code = "NIFTY"
exchange_code = "NSE"
product_type = "cash"


# get historical data for option premium for a particular strike price and underlying data
df = getDF(breeze.get_historical_data(interval,from_date,to_date,stock_code,exchange_code,product_type))

colsToDrop = ['stock_code', 'product_type', 'exchange_code','expiry_date','right','strike_price','count','open_interest',]
df =df.drop(columns =colsToDrop)
df['datetime'] = pd.to_datetime(df['datetime']).dt.normalize()
colsToFloat = ['open','close','high','low']
df[colsToFloat] = df[colsToFloat].apply(pd.to_numeric)
df['hc_diff'] = df['high'] - df['low']
df['oc_diff'] = df['open'] - df['close']
len(df)

2470

In [85]:
df.sort_values(by='datetime')
for i in range(1, len(df)):
    df.loc[i,'ocPrev_diff'] = df.loc[i, 'open'] -df.loc[i-1, 'close']
df = df.reindex(df.ocPrev_diff.abs().sort_values(ascending=False).index)
df.head(255)


,datetime,open,high,low,close,volume,hc_diff,oc_diff,ocPrev_diff
1653,2020-03-23,7945.70,8159.25,7583.60,7610.25,1940141559,575.65,335.45,-799.75
2133,2022-02-24,16548.90,16705.25,16203.25,16247.95,385954350,502.00,300.95,-514.35
1647,2020-03-13,9107.60,10159.40,8555.15,9955.20,450017974,1604.25,-847.60,-482.55
824,2016-11-09,8067.50,8476.20,8002.25,8432.00,1696067110,473.95,-364.50,-476.05
1646,2020-03-12,10039.95,10040.75,9508.00,9590.15,378117001,532.75,449.80,-418.45
...,...,...,...,...,...,...,...,...,...
1292,2018-10-04,10754.70,10754.70,10547.25,10599.25,1681161596,207.45,155.45,-103.55
1816,2020-11-14,12823.35,12828.70,12749.45,12780.25,791508963,79.25,43.10,103.40
2122,2022-02-09,17370.10,17477.15,17339.00,17463.80,229829643,138.15,-93.70,103.35
1663,2020-04-08,8688.90,9131.70,8653.90,8748.75,245044864,477.80,-59.85,-103.30


In [86]:
len(df[df.ocPrev_diff.abs() >400])

7

### write to csv

In [50]:
df.to_csv('file1.csv')

In [15]:
# For options
interval = "30minute" # "1minute", "5minute", "30minute" or as "1day"
from_date = "01-Apr-2023"
to_date = "28-Jun-2023"
stock_code = "NIFTY"
exchange_code = "NFO"
product_type = "options"
expiry_date="28-Jun-2023"
right="call"
strike_price="17000"
columnsToDrop = ['stock_code', 'product_type', 'exchange_code','expiry_date','right','strike_price','volume','open_interest',]

# get historical data for option premium for a particular strike price and underlying data
df_call = getDF(breeze.get_historical_data(interval,from_date,to_date,stock_code,exchange_code,product_type,expiry_date,right,strike_price))
# df =df.drop(columns =columnsToDrop)

df_put =  getDF(breeze.get_historical_data(interval,from_date,to_date,stock_code,exchange_code,product_type,expiry_date,"put",strike_price))

In [16]:
df_put

,datetime,stock_code,exchange_code,product_type,expiry_date,right,strike_price,open,high,low,close,volume,open_interest,count
0,2023-04-03 09:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,221.25,257,221.25,236.15,47400,1514700,0
1,2023-04-03 09:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,237.1,238.05,230.2,232.15,20450,1520750,1
2,2023-04-03 10:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,233.15,241.9,230.9,240.3,8550,1521550,2
3,2023-04-03 10:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,242,245,237.45,245,9050,1524850,3
4,2023-04-03 11:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,244.8,247.3,240.1,240.1,6700,1526250,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-06-28 13:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.75,.85,.65,.75,649900,5073850,762
763,2023-06-28 13:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.75,.85,.7,.8,599850,4980550,763
764,2023-06-28 14:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.8,.8,.25,.3,754500,4896500,764
765,2023-06-28 14:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.3,.3,.1,.15,759100,4854200,765


### get_option_chain_quotes

In [19]:
# For stock
t =breeze.get_option_chain_quotes(stock_code="ITC",
                    exchange_code="NFO",
                    product_type="options",
                    strike_price="400",
                    right="call")
getDF(t)

,exchange_code,product_type,stock_code,expiry_date,right,strike_price,ltp,ltt,best_bid_price,best_bid_quantity,...,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price,ltq,open_interest,chnge_oi,total_buy_qty,total_sell_qty
0,NFO,Options,ITC,28-Mar-2024,Call,400.0,19.35,07-Mar-2024 15:29:58,19.20,6400,...,22.47,30.10,7.30,4385600,413.55,1600,2475200.0,0.0,200000,219200
1,NFO,Options,ITC,25-Apr-2024,Call,400.0,24.35,07-Mar-2024 15:23:59,24.55,1600,...,14.32,34.85,13.85,83200,413.55,1600,172800.0,0.0,43200,104000
2,NFO,Options,ITC,30-May-2024,Call,400.0,0.00,,14.05,3200,...,-100.00,36.95,16.45,0,413.55,0,0.0,0.0,3200,0


In [21]:
def extract(t,colsToDrop,low=None,high=None):
    df =getDF(t)
    df=df[df['ltp'] > 0] 
    df=df[df['strike_price'] > low] if low else df
    df=df[df['strike_price'] < high] if high else df
    df =df.drop(columns =colsToDrop)
    return df

def get_opt_chn(right,low=None,high=None):
    t=breeze.get_option_chain_quotes(stock_code,exchange_code,product_type=product_type,expiry_date=expiry_date,right=right)
    return extract(t,colsToDrop,low,high)

In [22]:
# for index
stock_code = "NIFTY"
exchange_code = "NFO"
product_type = "options"
expiry_date = "28-Sep-2023" # put future expiry date
colsToDrop = ['stock_code', 'product_type', 'exchange_code']
low=17700
high=19900


df_call = get_opt_chn("call")
df_put = get_opt_chn("put")
df_op_chain= df_call[['expiry_date','strike_price','ltp']].merge(df_put[['strike_price','ltp']], on = 'strike_price')
df_op_chain=df_op_chain.rename(columns={"ltp_x": "call", "ltp_y": "put","strike_price":"strike","expiry_date":"expiry"})
df_op_chain['diff_premium'] =abs(df_op_chain['call'] - df_op_chain['put'])
df_op_chain['sum'] =df_op_chain['call'] + df_op_chain['put']
df_op_chain = df_op_chain.reindex(df_op_chain.diff_premium.sort_values().index)
type(df_op_chain)
df_op_chain.head(10)

KeyError: 'ltp'

In [8]:
df=df[df['ltp'] > 0] 
df['spot_price'] = df['spot_price'].astype('float')
# df['intrinsic_value'] = np.maximum(0,df['spot_price'] - df['strike_price'])
# df=df[df['intrinsic_value'] > 0] 
#df=df[df['strike_price'] > 18000] 
df['intrinsic_value'] =df['spot_price'] - df['strike_price']
df['time_value'] = df['ltp'] - df['intrinsic_value']
df[['spot_price','strike_price','ltp','intrinsic_value','time_value']]

NameError: name 'df' is not defined

### get_quotes

In [9]:
stock_code = "NIFTY"
exchange_code = "NSE"
t = breeze.get_quotes(stock_code=stock_code,
                    exchange_code=exchange_code)
df = getDF(t)
df=df[df['exchange_code'] == exchange_code]
df

,exchange_code,product_type,stock_code,expiry_date,right,strike_price,ltp,ltt,best_bid_price,best_bid_quantity,...,best_offer_quantity,open,high,low,previous_close,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price
0,NSE,,NIFTY,None,None,0.0,19778.3,26-Jul-2023 16:20:07,0.0,0,...,0,19733.35,19825.6,19716.7,19680.6,0.496428,0.0,0.0,0,None


In [89]:
columnsToDrop = ['exchange_code', 'expiry_date', 'right','product_type','strike_price']
df1 =df.drop(columns =columnsToDrop)
df1

,stock_code,ltp,ltt,best_bid_price,best_bid_quantity,best_offer_price,best_offer_quantity,open,high,low,previous_close,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price
0,NIFTY,19497.3,06-Jul-2023 16:21:40,0.0,0,0.0,0,19385.7,19512.2,19373.0,19398.5,0.509318,0.0,0.0,0,None


In [90]:
# bar plot
df.plot(kind='line',
        x='ltp',
        y='spot_price',
        color='green')
 

TypeError: no numeric data to plot